In [1]:
import sys
sys.path.append('./venv/lib/python3.6/site-packages/')

import yaml
import weibopy as wb


with open('./credentials.yml') as f:
    cred = yaml.load(f)

oauth = wb.OAuthHandler(**cred['weibo']).authorize()
api = wb.API(oauth=oauth)
# data, status = api.statuses.user_timeline()

Please load the link https://api.weibo.com/oauth2/authorize?response_type=code&client_id=3677147061&redirect_uri=https%3A%2F%2Fwww.greenpeace.org%2F&state=rBTAqYzsVSnpetYVhxKSXdjRA7mAqi to your browser and authorize weibo API access.
Enter the full callback URL: https://www.greenpeace.org/usa/?state=rBTAqYzsVSnpetYVhxKSXdjRA7mAqi&code=4acae937a0f6156bd8084503336d139c
UID: 1292378751
Access Token expires at: 2023/07/08 21:57:38


In [2]:
from utils import MongoWriter
from weibo_data import *
writer = MongoWriter(**cred['mongo'])

In [51]:
writer.get_since_id(collection='statuses')

4257970281382180

In [57]:
def crawl_statuses(since_id=0, insert: bool=False):
    records = list()
    skip_id = set()
    max_id = 0
    while True:
        data, _ = api.statuses.user_timeline(since_id=since_id, max_id=max_id)
        records = [s for s in [Status(x) for x in data['statuses']] if s.get('_id') not in skip_id]
        print(len(records))
        if records:
            for record in records:
                if record.get('_id') not in skip_id:
                    print(record.get('_id'), record.get('created_at'))
                    if insert:
                        record.write(writer=writer)
                    skip_id.add(record.get('_id'))
            else:
                max_id = record.get('_id')
        else:
            break

In [59]:
crawl_statuses(since_id=0)

5
4250426327276250 Wed Jun 13 12:27:48 +0800 2018
4257970281382180 Wed Jul 04 08:04:48 +0800 2018
4251242778027475 Fri Jun 15 18:32:05 +0800 2018
4248940779458347 Sat Jun 09 10:04:47 +0800 2018
4248766729509158 Fri Jun 08 22:33:09 +0800 2018
4
4247581435297686 Tue Jun 05 16:03:13 +0800 2018
4246117354311557 Fri Jun 01 15:05:29 +0800 2018
4243617021947926 Fri May 25 17:30:03 +0800 2018
4241343654479917 Sat May 19 10:56:29 +0800 2018
0


In [60]:
crawl_statuses(since_id=4243617021947926)

5
4250426327276250 Wed Jun 13 12:27:48 +0800 2018
4257970281382180 Wed Jul 04 08:04:48 +0800 2018
4251242778027475 Fri Jun 15 18:32:05 +0800 2018
4248940779458347 Sat Jun 09 10:04:47 +0800 2018
4248766729509158 Fri Jun 08 22:33:09 +0800 2018
2
4247581435297686 Tue Jun 05 16:03:13 +0800 2018
4246117354311557 Fri Jun 01 15:05:29 +0800 2018
0


In [64]:
for s_data in data['statuses']:
    status = Status(s_data)
    
    print(status.get('_id'), status.get('created_at'))
    print(len(status._data.keys()), list(status._data.keys()))
#     status.write(writer=writer) 

4250426327276250 Wed Jun 13 12:27:48 +0800 2018
32 ['_id', 'created_at', 'text', 'textLength', 'source', 'favorited', 'truncated', 'pic_urls', 'geo', 'is_paid', 'annotations', 'reposts_count', 'comments_count', 'attitudes_count', 'isLongText', 'multi_attitude', 'hide_flag', 'visible', 'biz_ids', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', 'hot_weibo_tags', 'text_tag_tips', 'userType', 'positive_recom_flag', 'content_auth', 'gif_ids', 'is_show_bulletin', 'comment_manage_info', 'uid']
4257970281382180 Wed Jul 04 08:04:48 +0800 2018
30 ['_id', 'created_at', 'text', 'textLength', 'source', 'favorited', 'truncated', 'pic_urls', 'geo', 'is_paid', 'reposts_count', 'comments_count', 'attitudes_count', 'isLongText', 'multi_attitude', 'hide_flag', 'visible', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', 'hot_weibo_tags', 'text_tag_tips', 'userType', 'positive_recom_flag', 'content_auth', 'gif_ids', 'is_show_bulletin', 'comment_manage_info', 'uid']
4251242778

In [47]:
data2, status = api.statuses.user_timeline(max_id=4248766729509158)

In [65]:
for s_data in data2['statuses']:
    status = Status(s_data)
    print(status.get('_id'), status.get('created_at'))
    print(len(status._data.keys()), list(status._data.keys()))

4250426327276250 Wed Jun 13 12:27:48 +0800 2018
32 ['_id', 'created_at', 'text', 'textLength', 'source', 'favorited', 'truncated', 'pic_urls', 'geo', 'is_paid', 'annotations', 'reposts_count', 'comments_count', 'attitudes_count', 'isLongText', 'multi_attitude', 'hide_flag', 'visible', 'biz_ids', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', 'hot_weibo_tags', 'text_tag_tips', 'userType', 'positive_recom_flag', 'content_auth', 'gif_ids', 'is_show_bulletin', 'comment_manage_info', 'uid']
4248766729509158 Fri Jun 08 22:33:09 +0800 2018
31 ['_id', 'created_at', 'text', 'textLength', 'source', 'favorited', 'truncated', 'pic_urls', 'geo', 'is_paid', 'reposts_count', 'comments_count', 'attitudes_count', 'isLongText', 'multi_attitude', 'hide_flag', 'visible', 'biz_ids', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', 'hot_weibo_tags', 'text_tag_tips', 'userType', 'positive_recom_flag', 'content_auth', 'gif_ids', 'is_show_bulletin', 'comment_manage_info', 'uid']

In [67]:
s_data

{'created_at': 'Sat May 19 10:56:29 +0800 2018',
 'id': 4241343654479917,
 'idstr': '4241343654479917',
 'mid': '4241343654479917',
 'can_edit': False,
 'text': '#地球守护者# 520微公益新媒体展！5月19日、20日，来北京751-D-Park炉区北广场，与酷酷的企鹅合影，打印专属明信片，绿色和平等你来玩！[太开心][太开心] http://t.cn/R2WJhBp \u200b',
 'textLength': 163,
 'source_allowclick': 0,
 'source_type': 2,
 'source': '<a href="http://weibo.com/" rel="nofollow">iPhone客户端</a>',
 'favorited': False,
 'truncated': False,
 'in_reply_to_status_id': '',
 'in_reply_to_user_id': '',
 'in_reply_to_screen_name': '',
 'pic_urls': [{'thumbnail_pic': 'http://wx1.sinaimg.cn/thumbnail/4d08227fly1frggnf598tj20ux0qok34.jpg'},
  {'thumbnail_pic': 'http://wx2.sinaimg.cn/thumbnail/4d08227fly1frggnfkuz1j20qo0qo41q.jpg'},
  {'thumbnail_pic': 'http://wx3.sinaimg.cn/thumbnail/4d08227fly1frggng0bihj20zk0qodne.jpg'},
  {'thumbnail_pic': 'http://wx2.sinaimg.cn/thumbnail/4d08227fly1frggngj6zsj20zk0qok0u.jpg'}],
 'thumbnail_pic': 'http://wx1.sinaimg.cn/thumbnail/4d08227fly1frggnf598t

In [68]:
'created_at', 'can_edit', 'text', 'textLength', 'source_allowclick',
'source_type', 'source', 'favorited', 'truncated', 'pic_urls', 'thumbnail_pic', 'bmiddle_pic', 'original_pic', 'geo',
'is_paid', 'mblog_vip_type', 'user', 'annotations', 'reposts_count', 'comments_count', 
'attitudes_count', 'pending_approval_count', 'isLongText', 'multi_attitude', 'hide_flag', 'mlevel',
'visible', 'biz_ids', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', 'hot_weibo_tags', 
'text_tag_tips', 'userType', 'extend_info', 'more_info_type', 'positive_recom_flag', 'content_auth', 'gif_ids',
'is_show_bulletin', 'comment_manage_info'

'id', 'idstr', 'mid', 'in_reply_to_status_id', 'in_reply_to_user_id',
'in_reply_to_screen_name',

dict_keys(['created_at', 'id', 'idstr', 'mid', 'can_edit', 'text', 'textLength', 'source_allowclick', 'source_type', 'source', 'favorited', 'truncated', 'in_reply_to_status_id', 'in_reply_to_user_id', 'in_reply_to_screen_name', 'pic_urls', 'thumbnail_pic', 'bmiddle_pic', 'original_pic', 'geo', 'is_paid', 'mblog_vip_type', 'user', 'annotations', 'reposts_count', 'comments_count', 'attitudes_count', 'pending_approval_count', 'isLongText', 'multi_attitude', 'hide_flag', 'mlevel', 'visible', 'biz_ids', 'biz_feature', 'page_type', 'hasActionTypeCard', 'darwin_tags', 'hot_weibo_tags', 'text_tag_tips', 'userType', 'extend_info', 'more_info_type', 'positive_recom_flag', 'content_auth', 'gif_ids', 'is_show_bulletin', 'comment_manage_info'])

In [71]:
data3, status = api.statuses.user_timeline(max_id=0)

In [73]:
data3

{'error': 'User requests out of rate limit!',
 'error_code': 10023,
 'request': '/2/statuses/user_timeline.json'}

In [72]:
for s_data in data3['statuses']:
    status = Status(s_data)
    print(status.get('_id'), status.get('created_at'))

KeyError: 'statuses'

In [33]:
c_data, _ = api.comments.show(id=4241343654479917)

In [43]:
for c in c_data['comments']:
    comment = Comment(c)
    print(comment.get('_id'), comment.get('created_at'), comment.get('reply_id'))
    comment.write(writer=writer, write_obj=True) 

4251210913585992 Fri Jun 15 16:25:28 +0800 2018 4241364990027955


ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4251210913585992 }
ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4241364990027955 }


4245533058418384 Thu May 31 00:23:43 +0800 2018 None


ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4245533058418384 }


4243933770763240 Sat May 26 14:28:42 +0800 2018 None


...

ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4241373853228872 }
ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4241358799683935 }


4241364990027955 Sat May 19 12:21:18 +0800 2018 None


ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4241364990027955 }


4241358799683935 Sat May 19 11:56:41 +0800 2018 None


ERROR:root:E11000 duplicate key error collection: weibo_data.comments index: _id_ dup key: { : 4241358799683935 }
ERROR:root:E11000 duplicate key error collection: weibo_data.users index: _id_ dup key: { : 3928609054 }


In [41]:
c_data2, _ = api.comments.show(id=4241343654479917, page=2)
for c in c_data2['comments']:
    comment = Comment(c)
    print(comment.get('_id'), comment.get('created_at'), comment.get('reply_id'))
    status.write(writer=writer) 

In [54]:
# data, status = api.comments.show(id=4250426327276250)

for s_data in data['comments']:
    print(s_data['id'], s_data['floor_number'], s_data['created_at'], s_data['status']['id'], s_data.get('reply_comment', {}).get('id'))
#     s_data['text']

4259206032691689 60 Sat Jul 07 17:55:13 +0800 2018 4250426327276250 None
4257895908341169 0 Wed Jul 04 03:09:15 +0800 2018 4250426327276250 4250620259670924
4257838387854399 59 Tue Jul 03 23:20:41 +0800 2018 4250426327276250 None
4254975343009732 58 Tue Jun 26 01:43:58 +0800 2018 4250426327276250 None
4254198877519865 57 Sat Jun 23 22:18:34 +0800 2018 4250426327276250 None
4252939264331013 56 Wed Jun 20 10:53:18 +0800 2018 4250426327276250 None
4251844676740213 55 Sun Jun 17 10:23:49 +0800 2018 4250426327276250 None
4251818151961743 54 Sun Jun 17 08:38:24 +0800 2018 4250426327276250 None
4251806529390153 53 Sun Jun 17 07:52:15 +0800 2018 4250426327276250 None
4251596248239347 51 Sat Jun 16 17:56:38 +0800 2018 4250426327276250 None
4251449976667290 50 Sat Jun 16 08:15:25 +0800 2018 4250426327276250 None
4251428078415332 49 Sat Jun 16 06:48:24 +0800 2018 4250426327276250 None
4251288319498696 48 Fri Jun 15 21:33:03 +0800 2018 4250426327276250 None
4251261500654057 47 Fri Jun 15 19:46:29 

In [69]:
data, _ = api.users.show(uid=2625796571)

In [70]:
data

{'id': 2625796571,
 'idstr': '2625796571',
 'class': 1,
 'screen_name': '璇在天空下的一米阳光',
 'name': '璇在天空下的一米阳光',
 'province': '14',
 'city': '1',
 'location': '山西 太原',
 'description': '',
 'url': 'http://weibo.com/u/2625796571',
 'profile_image_url': 'http://tvax3.sinaimg.cn/crop.0.0.1242.1242.50/9c8279dbly8frxb9e3rj4j20yi0yignd.jpg',
 'cover_image': 'http://ww2.sinaimg.cn/crop.0.16.980.300/9c8279dbjw1eai8mqtoj1j20r80fbdmt.jpg',
 'cover_image_phone': 'http://ww2.sinaimg.cn/crop.0.0.640.640.640/a1d3feabjw1ecatccqmkbj20hs0hsmzb.jpg',
 'profile_url': 'u/2625796571',
 'domain': '',
 'weihao': '',
 'gender': 'f',
 'followers_count': 0,
 'friends_count': 0,
 'pagefriends_count': 0,
 'statuses_count': 0,
 'video_status_count': 0,
 'favourites_count': 629,
 'created_at': 'Tue Feb 21 21:23:00 +0800 2012',
 'following': False,
 'allow_all_act_msg': True,
 'geo_enabled': True,
 'verified': False,
 'verified_type': 10,
 'remark': '',
 'insecurity': {'sexual_content': False},
 'status': {},
 'ptype': 0

In [ ]:
from weibo_data import *
from utils import MongoWriter


def main(cred):
    writer = MongoWriter(**cred['mongo'])
    oauth = wb.OAuthHandler(**cred['weibo']).authorize()
    api = wb.API(oauth=oauth)
    data, _ = api.statuses.user_timeline()
    for s_data in data['statuses']:
        status = Status(s_data)
        print(status)
        status.write(writer=writer)


if __name__ == '__main__':
    with open('./credentials.yml') as f:
        cred = yaml.load(f)
    main(cred=cred)

In [ ]:
api = wb.API(oauth)
data, status = api.statuses.user_timeline(count=100)

In [ ]:
statuses = data.get('statuses', [])

In [ ]:
for status in statuses:
    print(status.get('created_at'), status.get('id'))

In [ ]:
data, status = api.statuses.user_timeline(count=100, max_id=4248766729509158, trim_user=1)

In [ ]:
statuses2 = data.get('statuses', [])

In [ ]:
for status in statuses2:
    print(status.get('created_at'), status.get('id'))

In [ ]:
status = statuses2[2]
status

In [ ]:
status.keys()

In [ ]:
data, _ = api.statuses.show(id='4243617021947926')

In [ ]:
data.keys()

In [ ]:
data['user']

In [ ]:
data, _ = api.users.show(uid='1292378751')

In [ ]:
data

In [ ]:
help(api.users.show)

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url', 
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count', 
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled',
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel', 'verified_contact_name', 
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', 'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'mbtype', 'mbrank', 'block_word', 'block_app', 'credit_score', 'user_ability', 'urank', 'story_read_state', 'vclub_member'

In [ ]:
'screen_name', 'name', 'province', 'city', 'location', 'description', 'url', 'profile_image_url',
'cover_image', 'cover_image_phone', 'profile_url', 'domain', 'gender', 'followers_count', 'friends_count',
'pagefriends_count', 'statuses_count', 'video_status_count', 'favourites_count', 'created_at', 'geo_enabled', 
'verified', 'insecurity', 'ptype', 'allow_all_comment', 'avatar_large', 'avatar_hd', 'verified_reason', 
'verified_state', 'verified_level', 'verified_type_ext', 'has_service_tel','verified_contact_name',
'verified_contact_email', 'verified_contact_mobile', 'follow_me', 'like', \
            'like_me', 'online_status', 'bi_followers_count', 'lang', 'star', 'block_word', 'block_app', 'credit_score',\
            'user_ability', 'urank', 'story_read_state',

In [ ]:
data, _ = api.comments.show(id='4250426327276250')

In [ ]:
comments = data['comments']
len(comments)

In [ ]:
for c in comments:
    print(c['created_at'], c['id'])

In [ ]:
data2, _ = api.comments.show(id='4250426327276250', max_id='4250427878887380')

In [ ]:
comments = data2['comments']

for c in comments:
    print(c['created_at'], c['id'])